In [3]:
import pandas as pd
import numpy as np

In [4]:
titles = pd.read_csv("./data/title.basics.tsv.gz", sep='\t', header=0)

/var/folders/xm/nt85m0h948dgt0jbkz1yr62h0000gn/T/ipykernel_57193/2447817183.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  titles = pd.read_csv("./data/title.basics.tsv.gz", sep='\t', header=0)


In [254]:
titles.head(10)

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"
5,tt0000006,short,Chinese Opium Den,Chinese Opium Den,0,1894,\N,1,Short
6,tt0000007,short,Corbett and Courtney Before the Kinetograph,Corbett and Courtney Before the Kinetograph,0,1894,\N,1,"Short,Sport"
7,tt0000008,short,Edison Kinetoscopic Record of a Sneeze,Edison Kinetoscopic Record of a Sneeze,0,1894,\N,1,"Documentary,Short"
8,tt0000009,movie,Miss Jerry,Miss Jerry,0,1894,\N,45,Romance
9,tt0000010,short,Leaving the Factory,La sortie de l'usine Lumière à Lyon,0,1895,\N,1,"Documentary,Short"


In [26]:
movies = titles[titles['titleType']=='movie'].copy()

# Why doesn't this work?
# movies = movies[movies['startYear'] > 2000]

In [27]:
# elements of startYear are strings (csv files don't contain type information)
type(movies['startYear'].iloc[0])

# simple casting doesn't work. '\N' is used as special null value in the dataset
#movies['startYear'].astype(int)
#pd.to_numeric(movies['startYear'])

# replace '\N' with panda null value
movies.loc[movies['startYear']=='\\N', 'startYear'] = pd.NA

# Simple casting still doesn't work
# See http://pandas.pydata.org/pandas-docs/stable/user_guide/gotchas.html#support-for-integer-na
# movies['startYear'].astype(int)

# possible options
#pd.to_numeric(movies['startYear']) # --> creates float types 
#movies['startYear'].astype(pd.Int64Dtype()) # --> pandas experimental integer type that supports null values

# Other options could be to assign dummy values for null (e.g. -1) and do `astype(int)`. 
# But this can create other issues (for example, if the dummy value is actually a possible value the column can have)

movies['startYear'] = movies['startYear'].astype(pd.Int64Dtype())

In [359]:
movies = movies[movies['startYear'] > 2000]

### Transformation example on string column + `explode`

#### Count how many movies there are per genre

In [360]:
movies['genres'].head(20)

11636               Action,Crime
13081                Documentary
34800     Comedy,Fantasy,Romance
61112                      Drama
67486                      Drama
67664                      Drama
69151                      Drama
76052                Documentary
80549                     Horror
86791       Comedy,Horror,Sci-Fi
93928                      Drama
94103         Action,Crime,Drama
95140                Documentary
95593                         \N
98033       Comedy,Drama,Fantasy
101031           Biography,Drama
101475                        \N
102645               Documentary
106095            Comedy,Mystery
106657               Documentary
Name: genres, dtype: object

In [361]:
type(movies['genres'].iloc[0])

str

In [362]:
movies.loc[movies['genres']=='\\N', 'genres'] = pd.NA

# with apply

# lambda
movies['genres'].apply(lambda genre: genre.split(',') if pd.notna(genre) else pd.NA)

# function (does the same as the `apply` above)
def split_genre_string(genre: str) -> str:
    if pd.isnull(genre):
        return pd.NA
    else:
        return genre.split(',')

movies['genres'].apply(split_genre_string)

# built-in functionality
movies['genres'].str.split(',')

11636                  [Action, Crime]
13081                    [Documentary]
34800       [Comedy, Fantasy, Romance]
61112                          [Drama]
67486                          [Drama]
                       ...            
10169350                 [Documentary]
10169377                 [Documentary]
10169389                      [Comedy]
10169399                       [Drama]
10169409                 [Documentary]
Name: genres, Length: 305653, dtype: object

In [344]:
# comparison of apply vs. built-in functionality
%timeit -n 10 movies['genres'].apply(lambda genre: genre.split(',') if pd.notna(genre) else pd.NA) 
%timeit -n 10 movies['genres'].str.split(',')

204 ms ± 5.01 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
108 ms ± 1.32 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [363]:
movies['genres'] = movies['genres'].str.split(',')

In [364]:
movies['genres']

11636                  [Action, Crime]
13081                    [Documentary]
34800       [Comedy, Fantasy, Romance]
61112                          [Drama]
67486                          [Drama]
                       ...            
10169350                 [Documentary]
10169377                 [Documentary]
10169389                      [Comedy]
10169399                       [Drama]
10169409                 [Documentary]
Name: genres, Length: 305653, dtype: object

In [365]:
movies['genres'].explode()

11636            Action
11636             Crime
13081       Documentary
34800            Comedy
34800           Fantasy
               ...     
10169350    Documentary
10169377    Documentary
10169389         Comedy
10169399          Drama
10169409    Documentary
Name: genres, Length: 464203, dtype: object

In [366]:
movies_flat_genres = movies.explode(column='genres')
movies_flat_genres['genres'].value_counts()

genres
Drama          105590
Documentary     95062
Comedy          51707
Thriller        23284
Action          22288
Horror          21362
Romance         20628
Crime           14739
Biography       11792
Adventure       11569
Family           9933
Mystery          9477
History          8810
Music            8408
Fantasy          7366
Sci-Fi           6386
Animation        5879
Sport            5051
Musical          3023
War              2847
Adult            1731
News             1341
Western           916
Reality-TV        452
Talk-Show         150
Game-Show          17
Short               1
Name: count, dtype: int64

###TODO: maybe How many movies are assigned 0, 1 or multiple genres?

In [367]:
# 0 genres
missing_genre = (movies_flat_genres['genres']=='\\N')
movies_flat_genres[missing_genre].shape

# 1 genre
genre_count_per_movie = movies_flat_genres[~missing_genre].groupby('tconst').size()
genre_count_per_movie[genre_count_per_movie == 1].shape

# multiple genres
genre_count_per_movie[genre_count_per_movie > 1].shape

# option 2
movies['genres'].str.len()

11636       2
13081       1
34800       3
61112       1
67486       1
           ..
10169350    1
10169377    1
10169389    1
10169399    1
10169409    1
Name: genres, Length: 305653, dtype: object

### What's the movie with the highest rating?
### What's the movie with the most number of votes?
### What are the 10 movies with the most number of votes?

In [28]:
ratings = pd.read_csv("https://datasets.imdbws.com/title.ratings.tsv.gz", sep='\t', header=0)

In [29]:
# When all rows are non-null for a column, pandas can infer correct numeric types
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1351320 entries, 0 to 1351319
Data columns (total 3 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   tconst         1351320 non-null  object 
 1   averageRating  1351320 non-null  float64
 2   numVotes       1351320 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 30.9+ MB


In [30]:
movies_with_ratings = movies.merge(ratings, on='tconst')

In [371]:
movies_with_ratings[movies_with_ratings['numVotes'] > 20000]['averageRating'].max()

9.1

In [372]:
movies_with_ratings.loc[movies_with_ratings['numVotes'].idxmax()]

tconst                         tt0468569
titleType                          movie
primaryTitle             The Dark Knight
originalTitle            The Dark Knight
isAdult                                0
startYear                           2008
endYear                               \N
runtimeMinutes                       152
genres            [Action, Crime, Drama]
averageRating                        9.0
numVotes                         2780231
Name: 17184, dtype: object

In [373]:
movies_with_ratings.sort_values(by='numVotes', ascending=False).head(10)

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes
17184,tt0468569,movie,The Dark Knight,The Dark Knight,0,2008,\N,152,"[Action, Crime, Drama]",9.0,2780231
53254,tt1375666,movie,Inception,Inception,0,2010,\N,148,"[Action, Adventure, Sci-Fi]",8.8,2468126
21884,tt0816692,movie,Interstellar,Interstellar,0,2014,\N,169,"[Adventure, Drama, Sci-Fi]",8.7,1988554
51,tt0120737,movie,The Lord of the Rings: The Fellowship of the Ring,The Lord of the Rings: The Fellowship of the Ring,0,2001,\N,178,"[Action, Adventure, Drama]",8.8,1944643
185,tt0167260,movie,The Lord of the Rings: The Return of the King,The Lord of the Rings: The Return of the King,0,2003,\N,201,"[Action, Adventure, Drama]",9.0,1916434
51339,tt1345836,movie,The Dark Knight Rises,The Dark Knight Rises,0,2012,\N,164,"[Action, Drama, Thriller]",8.4,1775988
186,tt0167261,movie,The Lord of the Rings: The Two Towers,The Lord of the Rings: The Two Towers,0,2002,\N,179,"[Action, Adventure, Drama]",8.8,1729339
78228,tt1853728,movie,Django Unchained,Django Unchained,0,2012,\N,165,"[Drama, Western]",8.5,1634586
8881,tt0372784,movie,Batman Begins,Batman Begins,0,2005,\N,140,"[Action, Crime, Drama]",8.2,1534316
7887,tt0361748,movie,Inglourious Basterds,Inglourious Basterds,0,2009,\N,153,"[Adventure, Drama, War]",8.3,1524621


### How many movies don't have ratings?

In [380]:
# It looks like some movies are lost through the join. The join performed previously is an "inner" join
len(movies_with_ratings) == len(movies)

True

In [381]:
movies_with_ratings = movies.merge(ratings, on='tconst', how='left')
len(movies_with_ratings) == len(movies_with_ratings)

True

In [385]:
movies_with_ratings[movies_with_ratings['averageRating'].isnull()].shape

(139181, 11)

### Filter long-running movies

In [60]:
movies = pd.read_csv('./data/top_voted.tsv', sep='\t', header=0)

In [70]:
movies.loc[movies['runtimeMinutes']=='\\N', 'runtimeMinutes'] = pd.NA

# alternatively pd.to_numeric(movies['runtimeMinutes'])
movies['runtimeMinutes'] = movies['runtimeMinutes'].astype(pd.Int64Dtype())

movies[movies['runtimeMinutes'] > 120]

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0111161,movie,The Shawshank Redemption,The Shawshank Redemption,0,1994,\N,142,Drama
1,tt0468569,movie,The Dark Knight,The Dark Knight,0,2008,\N,152,"Action,Crime,Drama"
2,tt1375666,movie,Inception,Inception,0,2010,\N,148,"Action,Adventure,Sci-Fi"
3,tt0137523,movie,Fight Club,Fight Club,0,1999,\N,139,Drama
4,tt0109830,movie,Forrest Gump,Forrest Gump,0,1994,\N,142,"Drama,Romance"
...,...,...,...,...,...,...,...,...,...
9989,tt15680228,movie,Bheed,Bheed,0,2023,\N,124,"Drama,History"
9990,tt0099426,movie,Bullet in the Head,Dip huet gai tau,0,1990,\N,136,"Action,Crime,Drama"
9994,tt5456546,movie,Judwaa 2,Judwaa 2,0,2017,\N,145,"Action,Comedy"
9996,tt6793580,movie,Champions,Campeones,0,2018,\N,124,"Comedy,Drama,Family"


In [55]:
movies['runtimeMinutes'] = movies['runtimeMinutes'].astype(pd.Int64Dtype())
#pd.to_numeric(movies['runtimeMinutes'])

### Movie ratings
1. The mov

In [78]:
movies = movies.merge(ratings, on='tconst').sort_values('averageRating', ascending=False)

In [87]:
movies[movies['numVotes'] > 1000000].sort_values('averageRating')

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes
28,tt0499549,movie,Avatar,Avatar,0,2009,\N,162,"Action,Adventure,Fantasy",7.9,1360782
34,tt0120338,movie,Titanic,Titanic,0,1997,\N,194,"Drama,Romance",7.9,1246848
50,tt0371746,movie,Iron Man,Iron Man,0,2008,\N,126,"Action,Adventure,Sci-Fi",7.9,1097890
53,tt1431045,movie,Deadpool,Deadpool,0,2016,\N,108,"Action,Comedy",8.0,1088023
21,tt0848228,movie,The Avengers,The Avengers,0,2012,\N,143,"Action,Sci-Fi",8.0,1433072
...,...,...,...,...,...,...,...,...,...,...,...
31,tt0071562,movie,The Godfather Part II,The Godfather Part II,0,1974,\N,202,"Crime,Drama",9.0,1324649
10,tt0167260,movie,The Lord of the Rings: The Return of the King,The Lord of the Rings: The Return of the King,0,2003,\N,201,"Action,Adventure,Drama",9.0,1916679
1,tt0468569,movie,The Dark Knight,The Dark Knight,0,2008,\N,152,"Action,Crime,Drama",9.0,2780624
8,tt0068646,movie,The Godfather,The Godfather,0,1972,\N,175,"Crime,Drama",9.2,1950364


In [97]:
movies[movies['startYear'] > 2000].groupby('startYear').size()

startYear
2001    206
2002    218
2003    210
2004    247
2005    253
2006    300
2007    304
2008    306
2009    303
2010    291
2011    319
2012    304
2013    331
2014    358
2015    331
2016    347
2017    333
2018    355
2019    335
2020    238
2021    287
2022    291
2023    121
dtype: int64